In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion import Emotion
import mediapipe as mp

In [53]:
dirpath = "./"
filepath = ["영상1.mp4", "영상2.mp4"]
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [56]:
cap = cv2.VideoCapture(dirpath+filepath[1])
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    while(cap.isOpened()):
        ret, frame = cap.read()
        rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # 얼굴 영역 탐지는 어떤 모델을 사용하셔도 괜찮습니다
        # 예시로 사용한 모델의 경우 속도가 느려서 보유하신 모델을 사용하시면 됩니다! 
        results = face_detection.process(rgbframe)
        rgbframe.flags.writeable = True
        rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            bbox = results.detections[0].location_data.relative_bounding_box
            top = rgbframe.shape[0]*bbox.ymin
            bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
            left = rgbframe.shape[1]*bbox.xmin
            right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

            pre_emo, _, _ = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
            # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
            cv2.putText(frame, f"Emotion: {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
        cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)
        
        cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()